In [2]:
# (_.~" PRESENTATION "~._) 

'''
Ce code à pour but  de scrapper les meilleurs ventes sur le site officiel
de Nintendo. Il s'organise comme ceci :

- Import des librairies
- Déclarations (méthodes et variables)
- Code principal (scrapping)
- Nettoyage et export du DataFrame

'''

# (_.~" IMPORTS "~._) 

from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from datetime import datetime
import numpy as np
import pandas as pd
import re

# ---

# (_.~" METHODES "~._)  

# NETTOYAGE SIMPLE DES BALISES

def simpleClean(toClean) :
    '''Cette méthode sert à retirer les balises lors du scrapping 
    et peut servir (par exemple) pour les titres.
    
    Entrée type list :
    La méthode prend l'argument toClean.
    
    Fonctionnement :
    On applique le filtre to_clean grace au re.compile('<.*?>'). 
    Il est donc nécessaire d'importer "re" en amont de la méthode.
    
    Sortie type list :
    La méthode retourne la liste finalClean instanciée au début.
    '''
    # Instance de finalClean
    finalClean = []
    
    # Nettoyage de chaque entitée de la liste
    for thingToClean in toClean :
        to_clean = re.compile('<.*?>')
        toCleanStr = str(thingToClean)
        toCleanStr = re.sub(to_clean, '',toCleanStr)
        finalClean.append(toCleanStr)
    
    # Remplacement de l'ancienne liste par la liste clean
    return finalClean

# NETTOYAGE AVANCE POUR LES VENTES

def salesClean(toClean) :
    '''Cette méthode sert à retirer les balises lors du scrapping 
    et séléctionner uniquement les ventes réalisées.
    
    Entrée type list :
    La méthode prend l'argument toClean.
    
    Fonctionnement :
    On applique le filtre to_clean grace au re.compile('<.*?>'). 
    Il est donc nécessaire d'importer "re" en amont de la méthode.
    Il retire ensuite la fin de l'élément correspondant à :
    " million pcs." pour finalement le convertir en float.
    
    Sortie type list :
    La méthode retourne la liste finalClean instanciée au début.
    '''
    # Instance de finalClean
    finalClean = []
    
    # Nettoyage de chaque entitée de la liste et conversion
    for salesToClean in toClean :
        to_clean = re.compile('<.*?>')
        toCleanStr = str(salesToClean)
        toCleanStr = re.sub(to_clean, '',toCleanStr)
        finalClean.append(float(toCleanStr[:-13]))
    
    # Remplacement de l'ancienne liste par la liste clean
    return finalClean

# ---

# (_.~" DECLARATIONS VARIABLES "~._) 

'''

CONTEXTE :

Après observation, on peut voir que les chiffres de Nintendo 
sont modifiés fin mars, fin juin, fin septembre et fin décembre.
Notre étude se concentrera sur les dates de mars 2017 à fin 2022.
Nous allons définir les URLs de chaque trimestre.

On utilisera web.archive.org que l'on peut dissequer ainsi :
https://web.archive.org/web/ <= site utilisé pour retrouver les archives
20190923080858 <= date de l'archive au format YYYYMMDDHHHHHH
/https://www.nintendo.co.jp/ir/en/finance/software/ <= site concerné
index <= Console concernée. Ici "index" == Switch
.html <= fin de l'URL

NOTE : Il faudra faire attention à supprimer les lignes créées en Fev 2017
       dans le df, effectivement elle sera doublée car la Switch viens 
       juste d'être sortie
       
'''

# Instance du DataFrame principal

df = pd.DataFrame(columns=['Date','Hardware','Game','Sales'])

# Instance des listes pour le scrapping

hardwareList = ["index","wiiu","3ds","wii","ds"]
yearList = ["2017","2018","2019","2020","2021","2022","2023"]
monthList = ["02","05","08","11"]

# Déclaration des variables nécessaires aux URLs où la place 02 et 04
# sont réservés pour les dates et les hardwares

url01 = "https://web.archive.org/web/"
url03 = "15120000/https://www.nintendo.co.jp/ir/en/finance/software/"
url05 = ".html"

# Variables nécessaire à la création d'URL

urlList = []
url = ""

# Variable pour connaitre le total de page scrappé/restante
actualPage = 0
totalPage = len(hardwareList)*len(yearList)*len(monthList)

# ---

# (_.~" CODE "~._)  

'''
Le code boucle parmis les hardwares, les années puis les mois (déclarés plus haut). 
Chaque page créée sera scrappé puis les données seront ajoutées au DataFrame df.

Note : la date du DataFrame correspond à la mise à jour indiquée par Nintendo et pas
à la date de l'archive scrappé depuis e site "web.archive.org" .

'''

print("\n")
print("_ - _ SCRAPPING _ - _")
print("\n")
print("Scrap en cours :")
print("00 % Starting ...", end="\r")

for hardware in hardwareList:
    
    for year in yearList:
        
        for month in monthList:
            url = url01 + year + month + url03 + hardware + url05
            
            # Changement de hardwareTemp en "Switch" si "index"
            if hardware == "index":
                hardwareTemp = "switch"
            else:
                hardwareTemp = hardware
            
            # Pour savoir où on se situe en scrapping
            actualPage += 1
            actualPercent = int(actualPage/totalPage*100)
            print("\r",actualPercent,"% effectués (", hardwareTemp, year, month,")            ", end="\r")
            
            
            # (_.~" SCRAPPIIIIING TIME "~._)
   
            # Récupération et stockage du code HTML
            page = urlopen(url)
            soup = bs(page, "html.parser")
            
            # ---
    
            # oOoOoOo TEXT MINING DES DONNEES oOoOoOo
    
            # Récupération de la date de mis à Jour des données selon Nintendo
            date = soup.find('div', attrs = {'class' : 'sinf_date'}).string
    
            # Recupération de l'année et du mois de la mise à jour
            dateUpdateStr = date[8:-2]
            dateUpdate = datetime.strptime(dateUpdateStr, '%B %d, %Y')

            # Instance des listes contenant les titres et les ventes
            titles = []
            sales = []

            # Récupération des données
            titles = soup.findAll('p', attrs = {'class' : "sales_title"})
            sales = soup.findAll('p', attrs = {'class' : "sales_value"})
            # Nettoyage des données
            titles = simpleClean(titles)
            sales = salesClean(sales)
            
            # ---
    
            # oOoOoOo CREATION DU DATAFRAME oOoOoOo
            
            # Création des données à insérer dans les colonnes Date et Console
            tempDateList = [dateUpdate for i in range(len(titles))]
            tempHardwareList = [hardwareTemp for i in range(len(titles))]
    
            # Création du DataFrame contenant les lignes à ajouter
            dfSoup = pd.DataFrame(list(zip(tempDateList, tempHardwareList, titles, sales)), 
                          columns=['Date','Hardware','Game','Sales'])

            # Fusion des deux DataFrame
            df = pd.concat([df, dfSoup])
            

# ---

print("\r 100% terminé !                    ", end="\r")
print("\n")

# Total des données
xdf, ydf = df.shape
totalData = xdf * ydf

print(totalData, "données collectées. \n")

# (_.~" NETTOYAGE PRIMAIRE DU DATAFRAME "~._) 

print("_ - _ DATAFRAME _ - _")
print("\n")
print("Check avant export :")

# Reset de l'index
df.reset_index(drop=True, inplace=True)

# Doublons check avant suppression
doublons = df.duplicated().sum()
if doublons > 0:
    print(doublons," doublons présents dans le DataFrame.")
    df.drop_duplicates(inplace=True)
    doublons = df.duplicated().sum()
    if doublons > 0:
        print(doublons," doublons restent après suppression dans le DataFrame.")
    else:
        print("Tous les doublons ont été supprimés !")
else:
    print("Il n'y a aucun doublon dans le DataFrame.")
    
# NaN check
nanDfTotal = df.isna().sum().sum()
nanDf = df.isna().sum()
if nanDfTotal > 0:
    print("Données manquantes dans le DataFrame")
    print(nanDf)
else:
    print("Il n'y a aucune données manquante dans le DataFrame.")
    
# ---

# (_.~" EXPORT "~._) 

# Export du Dataframe dans un fichier CSV

print("\n")
print("Export en cours :")

csvName = 'nintendo_bestsellers_scrapping.csv'

df.to_csv(csvName, index=False)
print("Export réussi sous :", csvName)



_ - _ SCRAPPING _ - _


Scrap en cours :
 100% terminé !                    

5464 données collectées. 

_ - _ DATAFRAME _ - _


Check avant export :
561  doublons présents dans le DataFrame.
Tous les doublons ont été supprimés !
Il n'y a aucune données manquante dans le DataFrame.


Export en cours :
Export réussi sous : nintendo_bestsellers_scrapping.csv
